In [24]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import seaborn as sns
#import pandas_gbp
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)

class Covid19:
    df_cumulative_raw = pd.DataFrame()
    df_cumulative_by_country = pd.DataFrame()
    df_daily = pd.DataFrame()
    df_world_population = pd.DataFrame()
    df_confirmed = pd.DataFrame()
    df_deaths = pd.DataFrame()
    df_recovered = pd.DataFrame()
    
    COL_COUNTRY = "country"
    COL_DATE = "date"
    COL_CONFIRMED = "confirmed"
    COL_DEATHS = "deaths"
    COL_RECOVERED = "recovered"
    COL_PROVINCE = "province_state"
    COL_ACTIVE = 'active'
    COL_POPULATION = 'population'
    COL_DAILY_CASES = 'Daily Cases'
    COL_DAILY_RECOVERED = 'Daily Recovered'
    COL_DAILY_DEATHS = 'Daily Deaths'

    def __init__(self, datefilter: str = '2020-02-20'):
        self.data = []
        self.df_confirmed = self.fetch_covid19_cases('confirmed_cases') #this is a tablename
        self.df_deaths = self.fetch_covid19_cases(self.COL_DEATHS)
        self.df_recovered = self.fetch_covid19_cases('recovered_cases') #this is a tablename
        self.df_world_population = self.fetch_population()
        self.preprocess(datefilter)
        #self.calculate_new()

    def preprocess(self, datefilter: str):
        #Preprocess
        #remove canada as per https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6
        self.df_recovered = self.df_recovered[self.df_recovered[self.COL_COUNTRY]!='Canada']
        #join the confirmed, deaths and recovered data sets
        df_covid = self.covid_flip_datetocolumn(self.df_confirmed, self.COL_CONFIRMED).merge(
            right = self.covid_flip_datetocolumn(self.df_deaths, self.COL_DEATHS), 
            how='left',
            on=[self.COL_COUNTRY, self.COL_PROVINCE, self.COL_DATE]
        ).merge(
            right = self.covid_flip_datetocolumn(self.df_recovered, self.COL_RECOVERED),
            how='left',
            on=[self.COL_COUNTRY, self.COL_PROVINCE, self.COL_DATE]
        )
        #remove ship data  https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6
        ship_rows = df_covid[self.COL_COUNTRY].str.contains('Diamond Princess') | df_covid[self.COL_COUNTRY].str.contains('MS Zaandam')
        nan_rows = df_covid[self.COL_PROVINCE].isna()
        df_covid = df_covid[~(ship_rows)]
        df_covid = df_covid[~(nan_rows)]
        #Calculate active cases
        df_covid[self.COL_ACTIVE] = df_covid[self.COL_CONFIRMED] - df_covid[self.COL_DEATHS] - df_covid[self.COL_RECOVERED]
        #Join with world population
        df_covid_popl = df_covid.join(self.df_world_population, how='outer', on= self.COL_COUNTRY)
        #China doesn't have a summary row (province = null) so calculate

        #TODO: This should probably be only be done for the by_country dataframe not raw
        for var in [self.COL_CONFIRMED, self.COL_DEATHS, self.COL_RECOVERED, self.COL_ACTIVE]:
            df_covid_popl[var + '_per1m'] = df_covid_popl[var] * 1000000 / df_covid_popl[self.COL_POPULATION]
            df_covid_popl[var + '_per1k'] = df_covid_popl[var] * 100000 / df_covid_popl[self.COL_POPULATION]
        #df_covid_popl[self.COL_DATE] = df_covid_popl[self.COL_DATE].map(lambda x: datetime.strptime(x, "%m/%d/%y"))

        #df_covid_popl.dropna(subset=[self.COL_DATE], inplace=True)
        df_covid_popl = df_covid_popl[df_covid_popl[self.COL_DATE] > np.datetime64(datefilter)]

        self.df_cumulative_raw = df_covid_popl
        self.df_cumulative_by_country = self.df_cumulative_raw[self.df_cumulative_raw[self.COL_PROVINCE] == 'ALL']
        self.df_cumulative_by_country.drop([self.COL_PROVINCE], axis=1, inplace=True)

        #calculate daily numbers
        self.calculate_daily_nos()

    def fetch_covid19_cases(self, table):
        # Construct a BigQuery client object.
        client = bigquery.Client()
        query = """
            SELECT *, 
            FROM `bigquery-public-data.covid19_jhu_csse.{0}`
        """.format(table)
        df_confirmed_cases = client.query(query).to_dataframe()
        df_confirmed_cases.drop(['latitude', 'longitude', 'location_geom'], axis=1, inplace=True)
        df_confirmed_cases['province_state'] = df_confirmed_cases['province_state'].fillna('ALL')
        df_confirmed_cases.rename(columns={'country_region': self.COL_COUNTRY}, inplace=True)
        return df_confirmed_cases

    def fetch_population(self):
        client = bigquery.Client()
        query = """
            SELECT country_code, country_name as country, midyear_population as population 
            FROM `bigquery-public-data.census_bureau_international.midyear_population`
            where year = EXTRACT(year from (CURRENT_DATE()))
            order by midyear_population desc
        """
        df_world_population = client.query(query).to_dataframe()
        df_world_population[self.COL_COUNTRY][df_world_population[self.COL_COUNTRY] == 'United States'] = 'US'
        df_world_population = df_world_population.set_index(self.COL_COUNTRY)
        return df_world_population

    def covid_flip_datetocolumn(self, df_covid: pd.DataFrame, col_name: str):
        dates = df_covid.columns[2:] #excluding country and province
        df_temp = df_covid.melt(
            id_vars = [self.COL_COUNTRY, self.COL_PROVINCE], 
            value_vars = dates,
            var_name = self.COL_DATE,
            value_name = col_name
        )
        df_temp[self.COL_DATE] = df_temp[self.COL_DATE].map(lambda strtodate: strtodate.replace('_','/')[1:])
        df_temp[self.COL_DATE] = df_temp[self.COL_DATE].map(lambda x: datetime.strptime(x, "%m/%d/%y"))
        return df_temp
        
    # Calculates ratio of two countries numbes either confirmed cases, deaths, recovered
    def covid_ratio_by_country(self, col_name, source, dest):
        df_top = self.df_cumulative_by_country[[col_name, self.COL_DATE]][self.df_cumulative_by_country[self.COL_COUNTRY] == source] 
        df_down = self.df_cumulative_by_country[[col_name, self.COL_DATE]][self.df_cumulative_by_country[self.COL_COUNTRY] == dest] 
        df_ratio = df_top.merge(df_down, on='date')
        df_ratio['ratio'] = df_ratio[col_name + '_x'] / df_ratio[col_name + '_y'] 
        return df_ratio[[self.COL_DATE, 'ratio']]

    def calculate_daily_nos(self):
        # new cases 
        df_covid_popl_new = self.df_cumulative_by_country.groupby(
            [self.COL_DATE, self.COL_COUNTRY])[
                'confirmed', 'deaths', 'recovered', 'active', 'population', 
                'confirmed_per1m','deaths_per1m', 'recovered_per1m',
                'confirmed_per1k','deaths_per1k', 'recovered_per1k'
            ].sum().reset_index()
        temp = df_covid_popl_new.groupby(['country', 'date', ])[self.COL_CONFIRMED, self.COL_DEATHS, self.COL_RECOVERED]
        temp = temp.sum().diff().reset_index()
        mask = temp[self.COL_COUNTRY] != temp[self.COL_COUNTRY].shift(1)
        temp.loc[mask, self.COL_CONFIRMED] = np.nan
        temp.loc[mask, self.COL_DEATHS] = np.nan
        temp.loc[mask, self.COL_RECOVERED] = np.nan
        # renaming columns
        temp.columns = [self.COL_COUNTRY, self.COL_DATE, self.COL_DAILY_CASES, self.COL_DAILY_DEATHS, self.COL_DAILY_RECOVERED]
        # merging new values
        df_covid_popl_new = pd.merge(df_covid_popl_new, temp, on=['country', 'date'])
        # filling na with 0
        df_covid_popl_new = df_covid_popl_new.fillna(0)
        # fixing data types
        cols = [self.COL_DAILY_CASES, self.COL_DAILY_DEATHS, self.COL_DAILY_RECOVERED]
        df_covid_popl_new[cols] = df_covid_popl_new[cols].astype('int')
        # 
        df_covid_popl_new[self.COL_DAILY_CASES] = df_covid_popl_new[self.COL_DAILY_CASES].apply(lambda x: 0 if x<0 else x)
        self.df_daily = df_covid_popl_new


In [25]:
covid19 = Covid19()


In [23]:
covid19.df_cumulative_raw[covid19.df_cumulative_raw['country'] =='China']['province_state'].unique()

array(['Tibet', 'Macau', 'Qinghai', 'Zhejiang', 'Hong Kong', 'Shanghai',
       'Hunan', 'Heilongjiang', 'Xinjiang', 'Ningxia', 'Inner Mongolia',
       'Beijing', 'Jilin', 'Guangdong', 'Gansu', 'Tianjin', 'Guizhou',
       'Liaoning', 'Sichuan', 'Shanxi', 'Hainan', 'Yunnan', 'Shandong',
       'Chongqing', 'Guangxi', 'Jiangsu', 'Jiangxi', 'Hebei', 'Shaanxi',
       'Henan', 'Hubei', 'Anhui', 'Fujian'], dtype=object)

In [20]:
covid19.df_cumulative_by_country[covid19.df_cumulative_by_country['country'] =='China']

,country,date,confirmed,deaths,recovered,active,country_code,population,confirmed_per1m,confirmed_per1k,deaths_per1m,deaths_per1k,recovered_per1m,recovered_per1k,active_per1m,active_per1k


In [26]:
covid19.df_daily

,date,country,confirmed,deaths,recovered,active,population,confirmed_per1m,deaths_per1m,recovered_per1m,confirmed_per1k,deaths_per1k,recovered_per1k,Daily Cases,Daily Deaths,Daily Recovered
0,2020-02-21,Afghanistan,0.0,0.0,0.0,0.0,36643815.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0
1,2020-02-21,Albania,0.0,0.0,0.0,0.0,3074579.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0
2,2020-02-21,Algeria,0.0,0.0,0.0,0.0,42972878.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0
3,2020-02-21,Andorra,0.0,0.0,0.0,0.0,85635.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0
4,2020-02-21,Angola,0.0,0.0,0.0,0.0,32522339.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19576,2020-06-06,West Bank and Gaza,464.0,3.0,400.0,61.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,23
19577,2020-06-06,Western Sahara,9.0,1.0,6.0,2.0,652271.0,13.797946,1.533105,9.198631,1.379795,0.153311,0.919863,0,0,0
19578,2020-06-06,Yemen,482.0,111.0,23.0,348.0,29884405.0,16.128814,3.714312,0.769632,1.612881,0.371431,0.076963,13,0,0
19579,2020-06-06,Zambia,1089.0,7.0,912.0,170.0,17426623.0,62.490593,0.401684,52.333720,6.249059,0.040168,5.233372,0,0,0
